# Analyzing CIA Factbook Data Using SQL

This project is to analyze data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The data reference is from this [link](https://www.cia.gov/the-world-factbook/)

## Connect Jupytor Notebook to database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

## Review first 5 rows from table

The descriptions for some of the columns:

- **name** — the name of the country.
- **area**— the country's total area (both land and water).
- **area_land** — the country's land area in square kilometers.
- **area_water** — the country's waterarea in square kilometers.
- **population** — the country's population.
- **population_growth**— the country's population growth as a percentage.
- **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
- **death_rate** — the country's death rate, or the number of death per year per 1,000 people.

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

We will find:
* Minimum population
* Maximum population
* Minimum population growth
* Maximum population growth

In [5]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Exploring Outliers

We will find countries which have the maximum population and the minimum population

In [21]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts)
OR population = (SELECT MAX(population) FROM facts);

Done.


name,population
Antarctica,0
World,7256490011


As a result, we should recalculate while excluding the row of the whole world because it leads to misunderstand.

In [8]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


We will see that the value of maximum population is changed to 1367 millions. It is more accurate.

## Exploring Average Population and Area

We will find an average of population and an average of area and we should take care of discarding the row for the whole planet.

In [9]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts
WHERE name != 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


## Finding Densely Populated Countries

we will identify countries that have:
* above average values for population.
* below average values for area.

In [22]:
%%sql
SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name != 'World')
AND area < (SELECT AVG(area) FROM facts WHERE name != 'World');

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Country with the maximum population

In [23]:
%%sql
SELECT name, population
FROM facts
WHERE population = (SELECT MAX(population) FROM facts WHERE name != 'World');

Done.


name,population
China,1367485388


## Country with the maximum population growth

In [24]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts WHERE name != 'World');

Done.


name,population_growth
South Sudan,4.02


## Country with the highest ratios of water to land

In [26]:
%%sql
SELECT name, area_water, area_land, ROUND(CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT),2) AS ratio_water_to_land 
FROM facts
WHERE CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) = 
(SELECT MAX(CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT))
FROM facts);

Done.


name,area_water,area_land,ratio_water_to_land
British Indian Ocean Territory,54340,60,905.67


## Country with more water than land

In [27]:
%%sql
SELECT name, area_water, area_land, ROUND(CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT),2) AS ratio_water_to_land 
FROM facts
WHERE CAST(area_water AS FLOAT)/CAST(area_land AS FLOAT) > 1;

Done.


name,area_water,area_land,ratio_water_to_land
British Indian Ocean Territory,54340,60,905.67
Virgin Islands,1564,346,4.52


## Country with a higher death rate than birth rate

In [28]:
%%sql
SELECT name, death_rate, birth_rate
FROM facts
WHERE death_rate > birth_rate;

Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16


## Country with the highest population/area ratio

In [30]:
%%sql
SELECT name, population, area, ROUND(CAST(population AS FLOAT)/CAST(area AS FLOAT),2) AS pop_area_ratio
FROM facts
WHERE CAST(population AS FLOAT)/CAST(area AS FLOAT) = 
(SELECT MAX(CAST(population AS FLOAT)/CAST(area AS FLOAT))
FROM facts);

Done.


name,population,area,pop_area_ratio
Macau,592731,28,21168.96
